## Scenario 3 (Azure): Team using a remote MLflow server on Azure

This scenario demonstrates connecting to a remote MLflow Tracking Server hosted on Azure (e.g., Azure VM), with a production-ready backend database (Azure Database for PostgreSQL) and optional artifact storage on Azure Blob.

High-level architecture:
- Tracking server: Azure VM (Ubuntu)
- Backend store: Azure Database for PostgreSQL (Flexible Server)
- Artifact store: local disk on VM (for simplicity) or Azure Blob Storage (optional)

You’ll run the MLflow server on the VM, then connect from this notebook via its public IP/DNS.


### Connect to the remote MLflow server (from this notebook)
Fill in your VM public IP/DNS below.

**⚠️ Important:** After running the connection cell (Cell 3), if you encounter permission errors when logging artifacts, **restart your Python kernel** and re-run Cell 3. This ensures environment variables are properly set.


In [1]:
import mlflow

tracking_uri = "http://4.193.213.78:5000"
mlflow.set_tracking_uri(tracking_uri)
print(f"✓ Tracking URI: {mlflow.get_tracking_uri()}")

✓ Tracking URI: http://4.193.213.78:5000


In [2]:
import time
import requests

start = time.time()
response = requests.get("http://4.193.213.78:5000")
latency = time.time() - start
print(f"Server response time: {latency:.2f} seconds")

Server response time: 0.52 seconds


In [3]:
# Only get active experiments (faster)
experiments = mlflow.search_experiments(view_type=mlflow.entities.ViewType.ACTIVE_ONLY)

### List existing experiments
Confirm connectivity and see what’s already in the server.


In [4]:
mlflow.search_experiments()

[<Experiment: artifact_location='/home/azureuser/mlruns-artifacts/3', creation_time=1761927675828, experiment_id='3', last_update_time=1761927675828, lifecycle_stage='active', name='my-experiment-12', tags={}>,
 <Experiment: artifact_location='/home/azureuser/mlruns-artifacts/1', creation_time=1761924255313, experiment_id='1', last_update_time=1761924255313, lifecycle_stage='active', name='azure-experiment-1', tags={'mlflow.experimentKind': 'custom_model_development'}>,
 <Experiment: artifact_location='/home/azureuser/mlruns-artifacts/0', creation_time=1761924050477, experiment_id='0', last_update_time=1761924050477, lifecycle_stage='active', name='Default', tags={}>]

### Train a model and log to the remote server
Logs params, metrics, and a model artifact to the Azure-hosted MLflow server.


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-12")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/10/31 21:42:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


  → Using HTTP artifact repository (artifact_uri was local path)
🏃 View run serious-boar-472 at: http://4.193.213.78:5000/#/experiments/3/runs/b982acc8a129405ea945bad0b4fa9e73
🧪 View experiment at: http://4.193.213.78:5000/#/experiments/3


HTTPError: 404 Client Error: Not Found for url: http://4.193.213.78:5000/models/model.pkl. Response text: <!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>


### Work with the Model Registry
Create a client pointed at the remote server and register the model.


In [ ]:
from mlflow.tracking import MlflowClient

# Use the same tracking URI as set in Cell 3
client = MlflowClient(tracking_uri="http://4.193.213.78:5000")
client.search_registered_models()


In [ ]:
run = mlflow.search_runs(experiment_ids=["1"]).iloc[0]
run_id = run.run_id

mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name="azure-iris-classifier"
)


### Notes and troubleshooting
- Ensure Postgres allows connections from the VM and requires SSL.
- If the UI is unreachable, confirm VM NSG/ports and public IP/DNS.
- For artifacts on Azure Blob, ensure the MLflow server has valid credentials and your MLflow version supports the configured scheme.
- Use a managed secret store (e.g., Azure Key Vault) for credentials in production.

**Artifact directory permissions (on VM):**
If you encounter permission errors when the MLflow server tries to write artifacts, ensure the artifact directory is owned by `azureuser`:
```bash
sudo chown -R azureuser:azureuser /home/azureuser/mlruns-artifacts
```
The setup script (`setup-mlflow-server.sh`) handles this automatically, but you may need to run this manually if the directory was created differently.
